In [ ]:
import pandas as pd
import pvlib 

import matplotlib as plt

from bokeh.io import output_notebook
output_notebook()

import captest as pvc

# %load_ext autoreload
# %autoreload 2

In [ ]:
meas = pvc.CapData('meas')

In [ ]:
meas.load_data(path='../tests/data/', fname='nrel_data.csv', source='AlsoEnergy')

In [ ]:
meas.trans

In [ ]:
meas.set_reg_trans(poa='irr-poa-', w_vel='wind--', t_amb='temp--', power='')

In [ ]:
meas.filter_irr(500, 600)

Use all defaults
- RC_irr 60% percentile
- RC_temp mean
- RC_wind mean
- Calculates values on filtered data
- inplace

In [ ]:
meas.rep_cond()

Use all defaults, except inplace
- RC_irr 60% percentile
- RC_temp mean
- RC_wind mean
- Calculates values on filtered data
- not inplace- returns rep cond and doesn't write to RCs

In [ ]:
meas.rc

In [ ]:
meas.rc = None

In [ ]:
meas.rep_cond(inplace=False)

In [ ]:
print(meas.rc)

## Irradiance Balance Method
The irr_bal option replaces the aggregation methods specified with the func agrument with the irrRC_balanced function.  This function is intended to implement a strict interpratation of common contract language that specifies the reporting irradiance be determined by finding the irradiance that results in a balance of points within a +/- percent range of the reporting irradiance.  The irrRC_balanced function iterates to a solution for the reporting irradiance by calculating the irradiance that has 10 datpoints in the filtered dataset above it, then filtering for a percentage of points around that irradiance, calculating what percentile the reporting irradiance is in 

In [ ]:
meas.reset_flt()

In [ ]:
meas.filter_irr(500, 900)

In [ ]:
rep_irr = pvc.irrRC_balanced(meas.df_flt, 0.8, 1.2, irr_col=meas._CapData__get_poa_col(), plot=True)

In [ ]:
rep_irr[0]

In [ ]:
meas.df_flt.shape[0]

In [ ]:
rep_irr[1].shape[0]

In [ ]:
meas.rep_cond(irr_bal=True, perc_flt=20)

In [ ]:
meas.rc

## Reporting Conditions for Time Periods

The freq option can be used to calculate reporting conditions for groups of time like months or seasons.  Any standard aggregation method can be used to calcualte the reporting conditiosn for each group using the func or the irr_bal method can be used.

In [ ]:
pvsyst = pvc.CapData('pvsyst')

In [ ]:
pvsyst.load_data(load_pvsyst=True)

In [ ]:
pvsyst.trans

In [ ]:
pvsyst.set_reg_trans(power='real_pwr--', poa='irr-poa-', t_amb='temp-amb-', w_vel='wind--')

In [ ]:
pvsyst.filter_irr(500, 900)

In [ ]:
pvsyst.rep_cond(freq='M')

In [ ]:
pvsyst.rc

In [ ]:
pvsyst.rep_cond(freq='M', irr_bal=True, perc_flt=40, inplace=False)

remove time filtering, so rep_cond only either uses data in df_flt or calcs for freq groups
test period selection is duplicated in filter_time

func by default calcs 60th, mean, mean
- apply func to data passed
- apply func to time period centered around test date
- if using pvsyst and test date is near beginning of year wrap the data
- apply irr_bal to data passed; not implemented currently
- apply irr_bal to test period; not implemented currently

calculate rcs for groups of time
- apply function to groupby
- iterate over groups and apply irr_balance

RC calc approaches
- calc all three from the same dataset
- calc RC_irr filter and calculate temp and wind
- irr_balance- iteratively calc RC_irr and then calc temp and wind

In [ ]:
pvsyst.rep_cond(freq='BQ-NOV')

In [ ]:
pvsyst.rc

In [ ]:
pvsyst.rep_cond(freq='BQ-NOV', irr_bal=True, perc_flt=20)